In [9]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [10]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=options)

In [11]:
# Navigate to the website
url = "https://www.construction-europe.com/search?keyword=develon"
driver.get(url)
time.sleep(2)

In [12]:
# Click the cookie button
driver.find_element(By.CSS_SELECTOR, '#cookieConsent > div > div > div > div.col-12.col-md-auto.text-center.py-2.align-self-center > button').click()

In [13]:
req = urllib.request.Request(url)
sourcecode = urllib.request.urlopen(url).read()
soup = BeautifulSoup(sourcecode, "html.parser")

In [14]:
# Click the options
brand = driver.find_element(By.ID, 'BrandFilter_4')
driver.execute_script("arguments[0].click();", brand)

type = driver.find_element(By.ID, 'TypeFilter_1')
driver.execute_script("arguments[0].click();", type)

category = driver.find_element(By.ID, 'CategoryFilter_1')
driver.execute_script("arguments[0].click();", category)

# Apply the selected options
driver.find_element(By.CSS_SELECTOR, '#filterOverlay > div > div > div > button.btn.btn-primary.btn-lg.btn-ApplyFilter').click()

In [15]:
# Scrape article details
full_address = []

for i in range(2):
    articles = soup.find_all("div", class_="khl-article-block-headline font-weight-bold h4")
    for article in articles:
        address = article.find("a")["href"]
        if address != 'articleurl' and address not in full_address:  # 'articleurl'이 아니고 중복되지 않은 경우에만 추가
            full_address.append(address)
        print(full_address)
        
    # next_page_button = driver.find_element(By.CSS_SELECTOR, '#pagination > li.page-item.next > a').click()
    # time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")

full_address

['https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article']
['https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article', 'https://www.DemolitionandRecycling.media/news/develon-fleet-helps-clear-former-innocenti-factory-plant/8029288.article']
['https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article', 'https://www.DemolitionandRecycling.media/news/develon-fleet-helps-clear-former-innocenti-factory-plant/8029288.article', 'https://www.construction-europe.com/news/hyundai-rebrands-and-reveals-three-new-excavators/8029264.article']
['https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article', 'https://www.DemolitionandRecycling.media/news/develon-fleet-helps-clear-former-innocenti-factory-plant/8029288.article', 'https://www.construction-europe.com

['https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article',
 'https://www.DemolitionandRecycling.media/news/develon-fleet-helps-clear-former-innocenti-factory-plant/8029288.article',
 'https://www.construction-europe.com/news/hyundai-rebrands-and-reveals-three-new-excavators/8029264.article',
 'https://www.construcaolatinoamericana.com/news/hd-hyundai-infracore-chile-avanca-o-que-vem-com-develon/8029134.article',
 'https://www.construccionlatinoamericana.com/news/hd-hyundai-infracore-chile-adelanta-lo-que-viene-con-develon/8029133.article',
 'https://www.international-construction.com/news/construction-industry-takes-over-washington-d.c.-as-sector-highlights-us-1-trillion-value/8029089.article',
 'https://www.DemolitionandRecycling.media/news/hd-hyundai-infracore-targets-middle-east-and-africa-with-develon/8029040.article',
 'https://www.international-construction.com/news/develon-launches-new-6-tonne-stage-v-mini-excavators

In [16]:
article_data = []
for article_link in full_address:
    driver.get(article_link)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    release_date = ""
    title = ""
    content = ""
   
    # Scrape release date
    release_date_elem = soup.find("span", class_="PubDate")
    release_date = release_date_elem.text.strip() if release_date_elem else ""
            
    # Scrape news title
    title_elem = soup.find("h1", class_="khl-article-page-title")
    title = title_elem.text.strip() if title_elem else ""

    # Scrape article content
    first = soup.find("div", class_="col-12 khl-article-page-standfirst").text
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])

    # Check if content is empty
    if not content:
        continue

    # Append article data to the list
    article_data.append({"Date": release_date, "Title": title, "Content": content})
driver.quit()

In [17]:
# Create DataFrame from article data
df = pd.DataFrame(article_data)
df

,Date,Title,Content
0,06 June 2023,Heavy wheeled loaders: The workhorse of the co...,Heavy wheeled loaders have a home in almost ev...
1,30 May 2023,Develon fleet helps clear former Innocenti fac...,"The company, which announced its rebrand earli..."
2,30 May 2023,Hyundai rebrands and reveals three new excavators,It has been announced that HD Hyundai Construc...
3,24 May 2023,HD Hyundai Infracore Chile avança o que vem co...,"\nHD Hyundai Infracore Chile S.A. (HDIC), real..."
4,22 May 2023,HD Hyundai Infracore Chile adelanta lo que vie...,"\nHD Hyundai Infracore Chile S.A. (HDIC), real..."
5,18 May 2023,"Construction industry takes over Washington, D...",Heavy equipment manufacturers have been displa...
6,,HD Hyundai Infracore targets Middle East and A...,The announcement comes after the company poste...
7,10 May 2023,Develon launches new 6-tonne Stage V mini-exca...,\n\nDevelon has launched the new DX62R-7 and D...
8,03 May 2023,Digital construction site steals show on Samot...,"\n\n\n\n\nMachine control, remote control, aut..."
9,03 May 2023,Existe um limite para o quanto o equipamento d...,A automação de equipamentos de construção já e...


In [18]:
# Save DataFrame to CSV
df.to_csv("doosan_article_final.csv", index=False)